# Exploring the data
author: Nora Maleki (noora.maleki@gmail.com)
March 2022

## Import libraries

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Load datasets

In [53]:
FILE_PATH = r'../../data/01_main'

In [54]:
dataset = pd.read_csv(f'{FILE_PATH}/01_coordinates_integrated.csv', index_col=[0])
all_buildings = pd.read_csv(f'{FILE_PATH}/02_all_buildings_list.csv', index_col=[0])
avatar_info = pd.read_csv(f'{FILE_PATH}/03_avatar_info_list.csv', index_col=[0])
print('Done!')

Done!


In [55]:
dataset.head()

,SubjectID,ParticipantPosition_x,ParticipantPosition_z,PointerPosition_x,PointerPosition_z,Angle,BuildingPositionLocal_x,BuildingPositionLocal_y,BuildingPositionLocal_z,PointingTargetId,...,meaningful,meaningfulBuilding,Trials,TrialID,ImageName,TimeDelta,RT,TimeOut,TimeStampBegin,TimeStampEnd
0,365,-33.362625,-190.912018,-33.590775,-190.709625,87.005028,255.062286,-1.663044,-18.984482,1,...,True,Meaningful,0,29,01_CmA,0 days 00:00:07.635420416,7.635421,False,2021-11-18 10:26:39.660960000,2021-11-18 10:26:47.296380416
1,365,118.412003,-62.941864,118.647705,-63.044891,44.100197,255.062286,-1.663044,-18.984482,1,...,True,Meaningful,0,102,01_CmA,0 days 00:00:03.853917952,3.853918,False,2021-11-18 10:34:18.716167680,2021-11-18 10:34:22.570085632
2,365,257.929993,-33.097687,257.598694,-32.840240,47.522392,255.062286,-1.663044,-18.984482,1,...,True,Meaningful,0,155,01_CmA,0 days 00:00:03.418430208,3.418430,False,2021-11-18 10:40:28.512252160,2021-11-18 10:40:31.930682368
3,365,-242.933212,39.037964,-242.879013,38.953239,1.100918,255.062286,-1.663044,-18.984482,1,...,True,Meaningful,0,223,01_CmA,0 days 00:00:04.257166592,4.257167,False,2021-11-18 10:47:55.246826496,2021-11-18 10:47:59.503993088
4,365,-320.974487,77.197166,-320.853760,77.009575,29.176662,255.062286,-1.663044,-18.984482,1,...,True,Meaningful,0,303,01_CmA,0 days 00:00:02.474542336,2.474542,False,2021-11-18 11:08:19.997678080,2021-11-18 11:08:22.472220416


In [56]:
all_buildings.head()

,BuildingID,BuildingName,BuildingCenterWorld.x,BuildingCenterWorld.y,BuildingCenterWorld.z,BuildingPositionGlobal.x,BuildingPositionGlobal.y,BuildingPositionGlobal.z,BuildingPositionLocal.x,BuildingPositionLocal.y,...,BuildingLocalScale.z,BuildingForwardDirection.x,BuildingForwardDirection.y,BuildingForwardDirection.z,BuildingUpDirection.x,BuildingUpDirection.y,BuildingUpDirection.z,BuildingRightDirection.x,BuildingRightDirection.y,BuildingRightDirection.z
0,59,Crane_59,-318.127991,26.896717,-167.823914,-313.499451,1.614944,-150.514664,-313.499451,1.614944,...,1.20745,2.423655e-01,-7.277503e-09,9.701850e-01,5.915870e-08,1.000000e+00,-7.277503e-09,0.970185,-5.915870e-08,-0.242366
1,58,BasketballCourt_58,104.339996,-0.440584,-45.689999,104.339996,-3.070000,-45.689999,104.339996,-3.070000,...,1.00000,-6.614817e-01,0.000000e+00,7.499613e-01,0.000000e+00,1.000000e+00,0.000000e+00,0.749961,0.000000e+00,0.661482
2,57,Construction_57,-71.695419,0.739893,-26.238297,-71.803406,0.551138,-26.717417,-0.028043,0.005596,...,1.00000,1.490116e-07,1.000000e+00,-1.639128e-07,7.768201e-01,-2.384186e-07,-6.297227e-01,0.629723,0.000000e+00,0.776820
3,48,TaskBuilding_48,-195.637009,7.350957,-130.434052,-195.637512,-0.000015,-130.434052,-195.637512,130.434067,...,1.00000,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000,0.000000e+00,0.000000
4,52,TaskBuilding_52,-247.185669,4.570368,54.489395,-247.203369,2.410477,54.642479,-247.203369,-54.642483,...,1.00000,1.147317e-01,-4.165431e-07,-9.933965e-01,2.397451e-08,1.000000e+00,-4.165431e-07,-0.993397,-2.397451e-08,-0.114732


In [57]:
avatar_info.head()

,AvatarID,AvatarName,AvatarCenterWorld.x,AvatarCenterWorld.y,AvatarCenterWorld.z,AvatarPositionGlobal.x,AvatarPositionGlobal.y,AvatarPositionGlobal.z,AvatarPositionLocal.x,AvatarPositionLocal.y,...,AvatarLocalScale.z,AvatarForwardDirection.x,AvatarForwardDirection.y,AvatarForwardDirection.z,AvatarUpDirection.x,AvatarUpDirection.y,AvatarUpDirection.z,AvatarRightDirection.x,AvatarRightDirection.y,AvatarRightDirection.z
0,2,02_Cma,-147.664993,3.072653,67.390167,-147.646744,2.345342,67.389999,-351.640015,6.398831,...,1.0,0.000000,0.0,1.000000,0.0,1.0,0.0,1.000000,0.0,0.000000
1,42,42_Sa,65.630707,-1.885010,-52.687607,65.613266,-2.612321,-52.674149,-138.380005,1.441168,...,1.0,-0.605179,0.0,-0.796090,0.0,1.0,0.0,-0.796090,0.0,0.605179
2,36,36_Sa,162.001144,-0.394517,133.271271,162.000000,-1.121828,133.300003,-41.993271,2.931661,...,1.0,0.664752,0.0,0.747064,0.0,1.0,0.0,0.747064,0.0,-0.664752
3,38,38_Sa,150.833649,-1.489311,-43.709518,150.816269,-2.300489,-43.692154,-53.176998,1.753000,...,1.0,0.711973,0.0,0.702207,0.0,1.0,0.0,0.702207,0.0,-0.711973
4,41,41_Sa,92.128632,0.218283,38.293304,92.163269,-0.467581,38.295853,-111.830002,3.585908,...,1.0,0.743769,0.0,-0.668437,0.0,1.0,0.0,-0.668437,0.0,-0.743769


#### Glimpse at the data

In [58]:
dataset.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3024 entries, 0 to 3023
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SubjectID                 3024 non-null   int64  
 1   ParticipantPosition_x     3024 non-null   float64
 2   ParticipantPosition_z     3024 non-null   float64
 3   PointerPosition_x         3024 non-null   float64
 4   PointerPosition_z         3024 non-null   float64
 5   Angle                     3024 non-null   float64
 6   BuildingPositionLocal_x   3024 non-null   float64
 7   BuildingPositionLocal_y   3024 non-null   float64
 8   BuildingPositionLocal_z   3024 non-null   float64
 9   PointingTargetId          3024 non-null   int64  
 10  PointingTargetIndex       3024 non-null   int64  
 11  PointingTargetName        3024 non-null   object 
 12  PointingTargetBuildingId  3024 non-null   int64  
 13  PointingTargetCondition   3024 non-null   object 
 14  Context 

In [59]:
dataset.describe()

,SubjectID,ParticipantPosition_x,ParticipantPosition_z,PointerPosition_x,PointerPosition_z,Angle,BuildingPositionLocal_x,BuildingPositionLocal_y,BuildingPositionLocal_z,PointingTargetId,PointingTargetIndex,PointingTargetBuildingId,avatarID,Trials,TrialID,RT
count,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000
mean,5949.222222,-2.380455,14.179977,-2.105650,14.240982,49.312632,-0.507066,-0.480803,3.461617,28.521495,55.543651,29.076058,28.521495,55.543651,168.500000,7.694524
std,2679.017027,174.526182,137.780203,174.422929,137.616658,43.940893,201.542360,3.702814,128.275646,16.143405,32.306260,16.729943,16.143405,32.306260,97.010457,5.776413
min,365.000000,-326.886047,-191.062988,-327.157532,-191.119629,0.000000,-431.064392,-6.542114,-256.052216,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.955793
25%,4176.000000,-121.389214,-131.491875,-114.630356,-128.914780,13.787071,-153.392502,-2.357716,-91.287842,15.000000,28.000000,14.000000,15.000000,28.000000,84.750000,3.419546
50%,6642.000000,-15.471565,33.366211,1.953339,33.455299,36.979969,-3.306354,-1.083273,18.519926,29.000000,56.000000,29.000000,29.000000,56.000000,168.500000,5.872636
75%,7842.000000,123.511269,135.194962,137.915379,135.195560,72.340826,151.605972,0.727402,100.204834,43.000000,84.000000,43.000000,43.000000,84.000000,252.250000,9.962899
max,9472.000000,366.834778,235.882217,366.869934,235.829254,179.972031,401.828735,18.361389,293.247559,56.000000,111.000000,59.000000,56.000000,111.000000,336.000000,30.010961


In [60]:
all_buildings.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 58
Data columns (total 40 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   BuildingID                       59 non-null     int64  
 1   BuildingName                     59 non-null     object 
 2   BuildingCenterWorld.x            59 non-null     float64
 3   BuildingCenterWorld.y            59 non-null     float64
 4   BuildingCenterWorld.z            59 non-null     float64
 5   BuildingPositionGlobal.x         59 non-null     float64
 6   BuildingPositionGlobal.y         59 non-null     float64
 7   BuildingPositionGlobal.z         59 non-null     float64
 8   BuildingPositionLocal.x          59 non-null     float64
 9   BuildingPositionLocal.y          59 non-null     float64
 10  BuildingPositionLocal.z          59 non-null     float64
 11  BuildingRotationGlobal.x         59 non-null     float64
 12  BuildingRotationGlobal.y

In [61]:
all_buildings.describe()

,BuildingID,BuildingCenterWorld.x,BuildingCenterWorld.y,BuildingCenterWorld.z,BuildingPositionGlobal.x,BuildingPositionGlobal.y,BuildingPositionGlobal.z,BuildingPositionLocal.x,BuildingPositionLocal.y,BuildingPositionLocal.z,...,BuildingLocalScale.z,BuildingForwardDirection.x,BuildingForwardDirection.y,BuildingForwardDirection.z,BuildingUpDirection.x,BuildingUpDirection.y,BuildingUpDirection.z,BuildingRightDirection.x,BuildingRightDirection.y,BuildingRightDirection.z
count,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,...,59.000000,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,59.000000,5.900000e+01,59.000000
mean,30.000000,-6.712761,6.600042,2.499345,-6.834191,-0.430375,3.434584,-5.617659,-8.833987,-5.236339,...,1.003516,8.298783e-02,5.084729e-02,8.181360e-02,8.423508e-03,8.474576e-01,-2.704659e-02,0.097501,-2.456988e-08,-0.052710
std,17.175564,200.838912,5.763984,129.512759,201.222465,3.723653,128.471093,201.039792,125.917491,23.581244,...,0.027008,5.699982e-01,3.905667e-01,7.234338e-01,2.884677e-01,3.626321e-01,2.667254e-01,0.769663,1.508120e-07,0.642155
min,1.000000,-424.961121,-0.791109,-263.656067,-431.064392,-6.542114,-256.052216,-431.064392,-293.247589,-150.514664,...,1.000000,-9.997830e-01,-1.000000e+00,-9.999998e-01,-9.998724e-01,-2.384186e-07,-9.999999e-01,-1.000000,-4.172325e-07,-0.999872
25%,15.500000,-160.035126,4.143510,-79.115856,-159.725418,-2.322878,-79.156292,-159.725418,-96.718616,-2.377509,...,1.000000,-6.326982e-01,-3.920658e-07,-7.404863e-01,-8.994379e-08,1.000000e+00,-3.837909e-07,-0.743891,-1.158900e-07,-0.637716
50%,30.000000,-7.465199,5.415113,-0.542253,-7.537856,-0.946640,-0.364021,-3.306354,-3.070000,-1.083276,...,1.000000,8.662515e-07,-1.097826e-07,3.576279e-07,2.518646e-08,1.000000e+00,-1.729682e-07,0.437528,-2.901228e-08,-0.149642
75%,44.500000,136.813644,7.507464,96.738884,138.714348,0.851020,96.718609,138.714348,63.729702,0.585904,...,1.000000,5.897807e-01,0.000000e+00,7.563284e-01,1.516654e-07,1.000000e+00,0.000000e+00,0.768934,8.456120e-08,0.655384
max,59.000000,401.768616,31.293985,288.053894,401.828735,18.361389,293.247559,401.828735,256.052246,18.361364,...,1.207450,9.409345e-01,1.000000e+00,1.000000e+00,9.893244e-01,1.000000e+00,9.417416e-01,1.000000,5.364418e-07,0.999783


In [62]:
avatar_info.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 55
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   AvatarID                       56 non-null     int64  
 1   AvatarName                     56 non-null     object 
 2   AvatarCenterWorld.x            56 non-null     float64
 3   AvatarCenterWorld.y            56 non-null     float64
 4   AvatarCenterWorld.z            56 non-null     float64
 5   AvatarPositionGlobal.x         56 non-null     float64
 6   AvatarPositionGlobal.y         56 non-null     float64
 7   AvatarPositionGlobal.z         56 non-null     float64
 8   AvatarPositionLocal.x          56 non-null     float64
 9   AvatarPositionLocal.y          56 non-null     float64
 10  AvatarPositionLocal.z          56 non-null     float64
 11  AvatarRotationGlobal.x         56 non-null     float64
 12  AvatarRotationGlobal.y         56 non-null     float

In [63]:
avatar_info.describe()

,AvatarID,AvatarCenterWorld.x,AvatarCenterWorld.y,AvatarCenterWorld.z,AvatarPositionGlobal.x,AvatarPositionGlobal.y,AvatarPositionGlobal.z,AvatarPositionLocal.x,AvatarPositionLocal.y,AvatarPositionLocal.z,...,AvatarLocalScale.z,AvatarForwardDirection.x,AvatarForwardDirection.y,AvatarForwardDirection.z,AvatarUpDirection.x,AvatarUpDirection.y,AvatarUpDirection.z,AvatarRightDirection.x,AvatarRightDirection.y,AvatarRightDirection.z
count,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,...,56.0,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000
mean,28.500000,9.451360,-0.154723,8.906428,9.479183,-0.852688,8.787668,-194.514089,3.200802,-224.888190,...,1.0,0.152601,-0.000248,0.181139,-0.000041,0.999998,0.000191,0.181141,0.000098,-0.152601
std,16.309506,192.022522,2.171865,120.788425,191.980332,2.207004,120.734400,191.980334,2.207004,120.734400,...,0.0,0.596715,0.001783,0.777812,0.000304,0.000012,0.001847,0.777813,0.000539,0.596715
min,1.000000,-336.241150,-5.256408,-251.694504,-336.241333,-6.063000,-251.780014,-540.234619,-2.009511,-485.455872,...,1.0,-0.961716,-0.013202,-0.999927,-0.002276,0.999913,-0.003719,-0.999927,0.000000,-0.966854
25%,14.750000,-141.405403,-1.509524,-58.579079,-141.419785,-2.255122,-58.426651,-345.413055,1.798367,-292.102509,...,1.0,-0.241996,0.000000,-0.655529,0.000000,1.000000,0.000000,-0.655529,0.000000,-0.745289
50%,28.500000,2.890167,-0.333378,9.498009,2.883270,-1.067159,9.492935,-201.110001,2.986331,-224.182922,...,1.0,0.000000,0.000000,0.468661,0.000000,1.000000,0.000000,0.468661,0.000000,0.000000
75%,42.250000,147.657764,1.529099,111.972822,147.616764,0.888565,111.858717,-56.376500,4.942055,-121.817141,...,1.0,0.745289,0.000000,0.980278,0.000000,1.000000,0.000000,0.980299,0.000000,0.241999
max,56.000000,395.572296,3.703803,230.311523,395.563293,3.100000,230.184113,191.570007,7.153489,-3.491745,...,1.0,0.966854,0.001019,1.000000,0.000000,1.000000,0.013202,1.000000,0.003576,0.961709


Remove boolean variable ` AvatarPresence ` since it is redundant due to the existence of the boolean variable ` Agent `. It also contains all the values reversed, i.e., True is False and False, True.

In [64]:
dataset.drop(columns=['AvatarPresence'], inplace=True)

Check for the number of participants

In [65]:
subj_ids = dataset.SubjectID.unique()
print(f' Subject IDs: {subj_ids}\n',
      f'Number of participants: {len(subj_ids)}')

 Subject IDs: [ 365 3572 4176 5741 6642 7264 7842 8469 9472]
 Number of participants: 9


In [73]:
not_common_ids = list(set(all_buildings.BuildingID.unique())
                    # - set(dataset.PointingTargetBuildingId.unique())
                    - set(avatar_info.AvatarID.unique())
                  )
not_common_ids

[57, 58, 59]